In [1]:
# Libraries to import 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
city = 'Abbotsford' #Abbotsford or -
fav_cuisine = ["Indian"]
rest_name = ["An Indian Affair","Gian's Indian Cuisine"]
usid = 100

In [5]:
pref_1  = 'atmosphere'
pref_2  = 'food'
pref_3  = 'service'

In [5]:
d=pd.read_csv("/Final_data_submit(1).csv")
o_user=pd.read_csv("o_user.csv")
o_res=pd.read_csv("/o_res.csv")
data_p=pd.read_csv("data_p.csv")

In [6]:
if city != '-':
    l = list(d["Address"])
    city_list = []
    for i in l:
        j = []
        i = str(i)
        j = i.split(",")
        if len(j) > 1:
            city_list.append(j[-2].lstrip(" "))
        else:
            city_list.append(str(j).lstrip(" "))
    d["city"] = city_list
    fil = d[(d['city'] == city) |  (d.Name.isin(rest_name))]
else:
    def distance(lat,lon,destination1,destination2):
        lat1, lon1 = lat,lon
        lat2, lon2 = destination1,destination2
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        return d

    d['Distance'] = d.apply(lambda x: distance(lat,lon , x.Latitude, x.Longitude), axis=1)
    fil = d[(d['Distance'] < 10) | (d.Name.isin(rest_name))]

In [8]:
fil_nam = list(fil['Name'])

boo = o_res.title.isin(fil_nam)
fil_res = o_res[boo]

boo1 = o_user.Restaurant_id.isin(list(o_res.resId))
fil_user = o_user[boo1]

boo2 = data_p.Restaurant.isin(fil_nam)
fil_data = data_p[boo2]

In [10]:
d['Tags'].isnull().sum()

0

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
if city != '':
    l = list(d["Address"])
    city_list = []
    for i in l:
        j = []
        
        i = str(i)
        j = i.split(",")
        if len(j) > 1:
            city_list.append(j[-2].lstrip(" "))
        else:
            city_list.append(str(j).lstrip(" "))
    d["city"] = city_list
    fil = d[(d['city'] == city) |  (d.Name.isin(rest_name))]
else:
    def distance(lat,lon,destination1,destination2):
        lat1, lon1 = lat,lon
        lat2, lon2 = destination1,destination2
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        return d

    d['Distance'] = d.apply(lambda x: distance(lat,lon , x.Latitude, x.Longitude), axis=1)
    fil = d[(d['Distance'] < 10) | (d.Name.isin(rest_name))]
fil_nam = list(fil['Name'])

boo = o_res.title.isin(fil_nam)
fil_res = o_res[boo]

boo1 = o_user.Restaurant_id.isin(list(o_res.resId))
fil_user = o_user[boo1]

boo2 = data_p.Restaurant.isin(fil_nam)
fil_data = data_p[boo2]
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=1, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=1, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])
cos=cosine_similarity(tfv_matrix, tfv_matrix)
ind=pd.Series(df.index, index=df["Name"]).drop_duplicates()
def give_rec_rest(rest, sig=cos):
    idx=ind[rest]
    sig_scores=list(enumerate(sig[idx])) #makes it a list keeping index in consideration
    sig_scores=sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores=sig_scores[1:31]
    
    indi=[i[0] for i in sig_scores]
    return df.iloc[indi]
df["Tags"]=df["Tags"].replace("-"," ")
tfv_matrix1= tfv.fit_transform(df["Tags"])
cos1=cosine_similarity(tfv_matrix1,tfv_matrix1)
def give_rec_tag(resta, sig=cos1):
    idx = ind[resta]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:31]
    indi= [i[0] for i in sig_scores]
    return df.iloc[indi]
x = []
y = []
for i in range(len(rest_name)):
    df1 = give_rec_rest(rest_name[i],cos)
    x.extend(list(df1.Name))
    df2 = give_rec_tag(rest_name[i],cos1)
    y.extend(list(df2.Name))
z = x+y
boo3 = d.Name.isin(z)
df3 =d[boo3]
df3 =df3.sort_values(by = 'Rating_weighted', ascending= False)
df3 = df3.reset_index()
df3.head(3)
result1 = list(df3.Name)
result1=result1[:20]
# print(output)
# print(data)
# final_features = [np.array(int_features)]
# prediction = model.predict(final_features)
# output = round(prediction[0], 2)
# output = 5
data = d
data['Cuisine'] = data['Cuisine'].fillna(" ")
import re
def remove_punc(text):
    text = text.replace(',', ' ')
    return text

data['Cuisine'] = data['Cuisine'].apply(lambda x : remove_punc(x))
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

tfv_matrix = tfv.fit_transform(data['Cuisine'])  
df = pd.DataFrame(tfv_matrix[0].T.todense(), index=tfv.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
kmeans = KMeans(n_clusters=50,init='k-means++',random_state=0)
kmeans.fit(tfv_matrix)
data['clusters'] = kmeans.labels_
def cluster_predict(str_input):
    Y = tfv.transform(str_input)
    prediction = kmeans.predict(Y)
    return prediction
user = fil_user
res = fil_res
users_id = list(user['User_id'])
new_user = usid
fav_cus = fav_cuisine
if new_user not in users_id:
        print('new user')
        f = fav_cus
        prediction = cluster_predict(f)
        data1 =  data[data['clusters'] == prediction[0]]
        list2 = list(data1['Name'])
        result2 = list2[:20]
else:
    print('existing user')
    x = list(user[(user['User_id'] == new_user) & (user['Rating_id'] > 3.0) ]['Restaurant_id'])
    my_favorite_restaurants = [(res[res['resId'] == i]['title'].values)[0] for i in x]
    result2=my_favorite_restaurants[:3]
    cus = [(data[data['Name'] == i]['Cuisine'].values)[0] for i in result2[:3]]
    prediction = cluster_predict(cus)
    data1 =  data[data['clusters'] == prediction[0]]
    list2 = list(data['Name'])
    result2 += list2

result3 = []
result={}
if len(result3)==0:
    p = list(set(result1+result2))
    for i in p:
        if i in result1 and i in result2:
                    x = result1.index(i)
                    y = result2.index(i)
                    t = (0.4 * x) + (0.6 * y)
        elif i in result1:
                    x = result1.index(i)
                    t = 0.4 * x
        elif i in result2:
                        y = result2.index(i)
                        t = 0.6 * y

        result[t] = i

else:
        p = list(set(result1+result2))
        for i in p:
            x = 0
            y = 0
            z = 0
            if i in result1:
                x = result1.index(i)
            if i in result2:
                y = result1.index(i)
            if i in result3:
                z = result1.index(i)
            
            t = (0.3 * x) + (0.4 * y) + (0.3 * z)
            result[t] = i
#x = sorted(list(result.keys()))
x=list(result.keys())
results = [result[x[i]] for i in range(len(x))]

new user


In [15]:

if city != '':
    l = list(d["Address"])
    city_list = []
    for i in l:
        j = []
        
        i = str(i)
        j = i.split(",")
        if len(j) > 1:
            city_list.append(j[-2].lstrip(" "))
        else:
            city_list.append(str(j).lstrip(" "))
    d["city"] = city_list
    fil = d[(d['city'] == city) |  (d.Name.isin(rest_name))]
else:
    def distance(lat,lon,destination1,destination2):
        lat1, lon1 = lat,lon
        lat2, lon2 = destination1,destination2
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        return d

    d['Distance'] = d.apply(lambda x: distance(lat,lon , x.Latitude, x.Longitude), axis=1)
    fil = d[(d['Distance'] < 10) | (d.Name.isin(rest_name))]
fil_nam = list(fil['Name'])

boo = o_res.title.isin(fil_nam)
fil_res = o_res[boo]

boo1 = o_user.Restaurant_id.isin(list(o_res.resId))
fil_user = o_user[boo1]

boo2 = data_p.Restaurant.isin(fil_nam)
fil_data = data_p[boo2]
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=1, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=1, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])
cos=cosine_similarity(tfv_matrix, tfv_matrix)
ind=pd.Series(df.index, index=df["Name"]).drop_duplicates()
def give_rec_rest(rest, sig=cos):
    idx=ind[rest]
    sig_scores=list(enumerate(sig[idx])) #makes it a list keeping index in consideration
    sig_scores=sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores=sig_scores[1:31]
    
    indi=[i[0] for i in sig_scores]
    return df.iloc[indi]
df["Tags"]=df["Tags"].replace("-"," ")
tfv_matrix1= tfv.fit_transform(df["Tags"])
cos1=cosine_similarity(tfv_matrix1,tfv_matrix1)
def give_rec_tag(resta, sig=cos1):
    idx = ind[resta]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:31]
    indi= [i[0] for i in sig_scores]
    return df.iloc[indi]
x = []
y = []
for i in range(len(rest_name)):
    df1 = give_rec_rest(rest_name[i],cos)
    x.extend(list(df1.Name))
    df2 = give_rec_tag(rest_name[i],cos1)
    y.extend(list(df2.Name))
z = x+y
boo3 = d.Name.isin(z)
df3 =d[boo3]
df3 =df3.sort_values(by = 'Rating_weighted', ascending= False)
df3 = df3.reset_index()
df3.head(3)
result1 = list(df3.Name)
result1=result1[:20]
# print(output)
# print(data)
# final_features = [np.array(int_features)]
# prediction = model.predict(final_features)
# output = round(prediction[0], 2)
# output = 5
data = d
data['Cuisine'] = data['Cuisine'].fillna(" ")
import re
def remove_punc(text):
    text = text.replace(',', ' ')
    return text

data['Cuisine'] = data['Cuisine'].apply(lambda x : remove_punc(x))
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

tfv_matrix = tfv.fit_transform(data['Cuisine'])  
df = pd.DataFrame(tfv_matrix[0].T.todense(), index=tfv.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
kmeans = KMeans(n_clusters=50,init='k-means++',random_state=0)
kmeans.fit(tfv_matrix)
data['clusters'] = kmeans.labels_
def cluster_predict(str_input):
    Y = tfv.transform(str_input)
    prediction = kmeans.predict(Y)
    return prediction
user = fil_user
res = fil_res
users_id = list(user['User_id'])
new_user = usid
fav_cus = fav_cuisine
if new_user not in users_id:
        print('new user')
        f = fav_cus
        prediction = cluster_predict(f)
        data1 =  data[data['clusters'] == prediction[0]]
        list2 = list(data1['Name'])
        result2 = list2[:20]
else:
    print('existing user')
    x = list(user[(user['User_id'] == new_user) & (user['Rating_id'] > 3.0) ]['Restaurant_id'])
    my_favorite_restaurants = [(res[res['resId'] == i]['title'].values)[0] for i in x]
    result2=my_favorite_restaurants[:3]
    cus = [(data[data['Name'] == i]['Cuisine'].values)[0] for i in result2[:3]]
    prediction = cluster_predict(cus)
    data1 =  data[data['clusters'] == prediction[0]]
    list2 = list(data['Name'])
    result2 += list2

result3 = []
result={}
if len(result3)==0:
    p = list(set(result1+result2))
    for i in p:
        if i in result1 and i in result2:
                    x = result1.index(i)
                    y = result2.index(i)
                    t = (0.4 * x) + (0.6 * y)
        elif i in result1:
                    x = result1.index(i)
                    t = 0.4 * x
        elif i in result2:
                        y = result2.index(i)
                        t = 0.6 * y

        result[t] = i

else:
        p = list(set(result1+result2))
        for i in p:
            x = 0
            y = 0
            z = 0
            if i in result1:
                x = result1.index(i)
            if i in result2:
                y = result1.index(i)
            if i in result3:
                z = result1.index(i)
            
            t = (0.3 * x) + (0.4 * y) + (0.3 * z)
            result[t] = i
#x = sorted(list(result.keys()))
x=list(result.keys())
results = [result[x[i]] for i in range(len(x))]

new user


In [16]:
results

['An Indian Affair',
 'Greek Islands Restaurant III',
 'Brodeurs Bistro',
 'Noodle Road',
 'Sapphire Bistro Banquet',
 "Papa John's Pizza",
 "Gian's Indian Cuisine",
 'Taj Indian Bistro',
 'Restaurant 62',
 "Lou's Grill",
 'Cactus Club Cafe',
 'White Rock Indian Flavours',
 'Curry Express',
 'Greek Islands Restaurant',
 "Cheryl's Cafe",
 'Spice Kitchen The Indian Palate',
 'Bayside Grill',
 'Chakra Indian Restaurant']

In [17]:
result=pd.DataFrame({"Name":results})

In [18]:
result

,Name
0,An Indian Affair
1,Greek Islands Restaurant III
2,Brodeurs Bistro
3,Noodle Road
4,Sapphire Bistro Banquet
5,Papa John's Pizza
6,Gian's Indian Cuisine
7,Taj Indian Bistro
8,Restaurant 62
9,Lou's Grill


In [19]:
final_result=d.merge(result,on="Name")

In [20]:
final_result

,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,...,time_f,time_f1,Cuisine,Price,Restaurant,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters
0,Restaurant,Noodle Road,4.5,66.0,"[{'days': 'Sun', 'times': ['12:00 PM - 9:30 PM...",49.048660,-122.339240,"['All reviews ', 'noodles ', 'sushi ', 'asian ...","24-31940 South Fraser Way, Abbotsford, British...",+1 604-557-0600,4.5,4.5,4.5,4.0,4.153338,0.0,NaN,12.0,21.3,NaN,NaN,1.0,6.0,11.3,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,japanese sushi asian,highcost,Noodle Road,"best food,great food,good bathroom,great meal,...",NaN,"great food and atmosphere,little gem serving d...",not a good place,Noodle Road,NaN,NaN,"24-31940 South Fraser Way, Abbotsford, British...",(604) 557-0600,support@noddleroad.ca,http://www.noodleroad.ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,24:00,21:00|Monday,11:00,21:00|Tuesday,...","Sushi,Asian,Korean,Japanese,Lunch,Dinner,Veget...","Takeout,Seating,ParkingAvailable,WheelchairAcc...","noodles,sushi,asiandishes,complimentaryappetiz...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,2
1,Restaurant,Noodle Road,4.5,66.0,"[{'days': 'Sun', 'times': ['12:00 PM - 9:30 PM...",49.048660,-122.339240,"['All reviews ', 'noodles ', 'sushi ', 'asian ...","24-31940 South Fraser Way, Abbotsford, British...",+1 604-557-0600,4.5,4.5,4.5,4.0,4.153338,0.0,NaN,12.0,21.3,NaN,NaN,1.0,6.0,11.3,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,japanese sushi asian,highcost,Noodle Road,"best food,great food,good bathroom,great meal,...",NaN,"great food and atmosphere,little gem serving d...",not a good place,Mr India Restaurant,NaN,NaN,"31940 South Fraser Way, Abbotsford, British Co...",(604) 870-4898,NaN,https://mr-india-meats-restaurant.business.site/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Wednesday,10:00,22:00|Thursday,10:00,22:00|Fri...","Indian,Asian,Dinner,Lunch,","Seating,WheelchairAccessible",",best service,clean place,outstanding cake,hap...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,2
2,Restaurant,Noodle Road,4.5,66.0,"[{'days': 'Sun', 'times': ['12:00 PM - 9:30 PM...",49.048660,-122.339240,"['All reviews ', 'noodles ', 'sushi ', 'asian ...","24-31940 South Fraser Way, Abbotsford, British...",+1 604-557-0600,4.5,4.5,4.5,4.0,4.153338,0.0,NaN,12.0,21.3,NaN,NaN,1.0,6.0,11.3,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,japanese sushi asian,highcost,Noodle Road,"best food,great food,good bathroom,great meal,...",NaN,"great food and atmosphere,little gem serving d...",not a good place,Super Punjab Sweet House,NaN,NaN,"31940 South Fraser Way, Abbotsford, British Co...",(604) 504-2828,NaN,http://superpunjabsweethouseabbotsford.gastrob...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,10:00,19:00|Monday,10:00,19:00|Tuesday,...",",Lunch,Dinner,",NaN,",good variety samosa,many people,fantastic sam...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,2
3,Restaurant,Noodle Road,4.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=3, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])

In [23]:
cos=cosine_similarity(tfv_matrix, tfv_matrix)
ind=pd.Series(df.index, index=df["Name"]).drop_duplicates()
def give_rec_rest(rest, sig=cos):
    idx=ind[rest]
    sig_scores=list(enumerate(sig[idx])) #makes it a list keeping index in consideration
    print(sig_scores)
    sig_scores=sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores=sig_scores[1:31]
    
    indi=[i[0] for i in sig_scores]
    return df.iloc[indi]

In [24]:
df["Tags"]=df["Tags"].replace("-"," ")
tfv_matrix1= tfv.fit_transform(df["Tags"])
cos1=cosine_similarity(tfv_matrix1,tfv_matrix1)
def give_rec_tag(resta, sig=cos1):
    idx = ind[resta]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:31]
    indi= [i[0] for i in sig_scores]
    return df.iloc[indi]

In [25]:
x = []
y = []
for i in range(len(rest_name)):
  df1 = give_rec_rest(rest_name[i],cos)
  x.extend(list(df1.Name))
  df2 = give_rec_tag(rest_name[i],cos1)
  y.extend(list(df2.Name))

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.0), (75, 0.0), (76, 0.0), (77, 0.0), (78, 0.0), (79, 0.0), (80, 0.0), (81, 0.0), (82, 0.0), (83, 0.0), (84, 0.3236594863943274), (85, 0.3236594863943274), (86, 0.3236594863943274), (87, 0.

In [26]:
z = x+y
boo3 = d.Name.isin(z)
df3 =d[boo3]
df3 =df3.sort_values(by = 'Rating_weighted', ascending= False)
df3 = df3.reset_index()
df3.head(3)

,index,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,...,time_f,time_f1,Cuisine,Price,Restaurant,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters
0,435,Restaurant,Brodeurs Bistro,4.5,816.0,"[{'days': 'Sun - Wed', 'times': ['11:00 AM - 1...",49.066475,-122.380360,"['All reviews ', 'smoked meat ', 'jambalaya ',...","3550 Mt Lehman Road, Abbotsford, British Colum...",+1 604-746-5900,4.5,4.5,4.5,4.0,4.455660,0.0,3.0,11.0,22.0,NaN,NaN,4.0,6.0,11.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,american cajun creole bar,highcost,Brodeurs Bistro,"best great food,good spot lunch,authentic loui...",NaN,"breakfast is a winner good value,once again ...","terrible service,lousy service poor food,cust...",Brodeurs Bistro,North American French Cuisine Taking the best ...,NaN,"3550 Mt Lehman Road, Abbotsford, British Colum...",(604) 746-5900,NaN,http://www.bbistro.ca/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,11:00,22:00|Monday,11:00,22:00|Tuesday,...","American,Bar,Canadian,Cajun&Creole,Lunch,Dinne...","Takeout,OutdoorSeating,Seating,ParkingAvailabl...","smokedmeat,jambalaya,cornbread,beignets,cubans...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,1
1,395,Restaurant,Greek Islands Restaurant,4.5,589.0,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,mediterranean greek canadian,highcost,Greek Islands Restaurant,"great friendly staff,good price,good wine beer...",NaN,NaN,NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46
2,394,Restaurant,Greek Islands Restaurant,4.5,589.0,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,mediterranean greek canadian,highcost,Greek Islands Restaurant,"food excellent staff,special occasion food,awe...",NaN,"exquisite chinese culinary restaurant ,delicio...",NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https:/

In [27]:
fil=final_result

In [36]:
fil.No_of_Reviews.head()

0    66
1    66
2    66
3    66
4    17
Name: No_of_Reviews, dtype: int64

In [38]:
fil = fil.replace('}','', regex=True)


fil.service= fil["service"].astype('float')


fil["No_of_Reviews"] = fil["No_of_Reviews"].astype(int)

# fil["service"] = fil["service"].replace("}","")

v=fil["No_of_Reviews"]
m=fil["No_of_Reviews"].quantile(0.70)


R_food=fil["food"] 
C_food=fil["food"].mean()

R_service=fil["service"]        #  R_service=d["service"] 
C_service=fil["service"]        #  C_service=d["service"].mean()

R_atmos=fil["atmosphere"] 
C_atmos=fil["atmosphere"].mean()

R_val=fil["value"] 
C_val=fil["value"].mean()


fil["food_weighted_average"]=((R_food*v)+ (C_food*m))/(v+m)
fil["service_weighted_average"]=((R_service*v)+ (C_service*m))/(v+m)
fil["atmosphere_weighted_average"]=((R_atmos*v)+ (C_atmos*m))/(v+m)
fil["value_weighted_average"]=((R_val*v)+ (C_val*m))/(v+m)

fil["fsva_weighted_average"] = fil[["food_weighted_average","service_weighted_average","value_weighted_average","atmosphere_weighted_average"]].sum(axis = 1)


In [39]:
def rec_based_on_pref(pref_1,pref_2,pref_3):
    pref_1_col = pref_1+"_weighted_average"
    pref_2_col = pref_2+"_weighted_average"
    pref_3_col = pref_3+"_weighted_average"
    df_sorted_on_pref=fil.sort_values(by=[pref_1_col,pref_2_col,pref_3_col],ascending=False)
    return df_sorted_on_pref
    

In [40]:
final_rec = pd.DataFrame(rec_based_on_pref(pref_1,pref_2,pref_3))

In [41]:
final_rec

,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,...,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters,food_weighted_average,atmosphere_weighted_average,value_weighted_average,service_weighted_average,fsva_weighted_average
12,Restaurant,Greek Islands Restaurant,4.5,589,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"food excellent staff,special occasion food,awe...",NaN,"exquisite chinese culinary restaurant ,delicio...",NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46,4.243405,4.190680,4.236375,4.5,17.170460
13,Restaurant,Greek Islands Restaurant,4.5,589,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"great friendly staff,good price,good wine beer...",NaN,NaN,NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46,4.243405,4.190680,4.236375,4.5,17.170460
19,Restaurant,An Indian Affair,4.5,390,"[{'days': 'Sun - Thu', 'times': ['11:00 AM - 9...",49.115620,-122.679530,"['All reviews ', 'naan ', 'butter chicken ', '...","146-19653 Willowbrook Dr, British Columbia V2Y...",+1 604-539-8114,4.5,4.5,4.5,4.5,4.412136,0.0,4.0,11.0,21.3,NaN,NaN,5.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"food great food,best indian food,service good ...","little piece place,unbelievable recommend plac...","fabulous,great meal ,fantastic ,absolutely gor...",NaN,An Indian Affair,An Indian Affair is one of the Best Indian foo...,NaN,"146-19653 Willowbrook Dr, British Columbia V2Y...",(604) 539-8114,Anindianaffair@hotmail.com,http://www.anindianaffair.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media

In [42]:
final_rec.sort_values(by = "fsva_weighted_average", ascending = False)

,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,...,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters,food_weighted_average,atmosphere_weighted_average,value_weighted_average,service_weighted_average,fsva_weighted_average
12,Restaurant,Greek Islands Restaurant,4.5,589,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"food excellent staff,special occasion food,awe...",NaN,"exquisite chinese culinary restaurant ,delicio...",NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46,4.243405,4.190680,4.236375,4.5,17.170460
13,Restaurant,Greek Islands Restaurant,4.5,589,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"great friendly staff,good price,good wine beer...",NaN,NaN,NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46,4.243405,4.190680,4.236375,4.5,17.170460
17,Restaurant,Brodeurs Bistro,4.5,816,"[{'days': 'Sun - Wed', 'times': ['11:00 AM - 1...",49.066475,-122.380360,"['All reviews ', 'smoked meat ', 'jambalaya ',...","3550 Mt Lehman Road, Abbotsford, British Colum...",+1 604-746-5900,4.5,4.5,4.5,4.0,4.455660,0.0,3.0,11.0,22.0,NaN,NaN,4.0,6.0,11.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"best great food,good spot lunch,authentic loui...",NaN,"breakfast is a winner good value,once again ...","terrible service,lousy service poor food,cust...",Brodeurs Bistro,North American French Cuisine Taking the best ...,NaN,"3550 Mt Lehman Road, Abbotsford, British Colum...",(604) 746-5900,NaN,http://www.bbistro.ca/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-..

In [43]:
df3

,index,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,...,time_f,time_f1,Cuisine,Price,Restaurant,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters
0,435,Restaurant,Brodeurs Bistro,4.5,816.0,"[{'days': 'Sun - Wed', 'times': ['11:00 AM - 1...",49.066475,-122.380360,"['All reviews ', 'smoked meat ', 'jambalaya ',...","3550 Mt Lehman Road, Abbotsford, British Colum...",+1 604-746-5900,4.5,4.5,4.5,4.0,4.455660,0.0,3.0,11.0,22.0,NaN,NaN,4.0,6.0,11.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,american cajun creole bar,highcost,Brodeurs Bistro,"best great food,good spot lunch,authentic loui...",NaN,"breakfast is a winner good value,once again ...","terrible service,lousy service poor food,cust...",Brodeurs Bistro,North American French Cuisine Taking the best ...,NaN,"3550 Mt Lehman Road, Abbotsford, British Colum...",(604) 746-5900,NaN,http://www.bbistro.ca/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,11:00,22:00|Monday,11:00,22:00|Tuesday,...","American,Bar,Canadian,Cajun&Creole,Lunch,Dinne...","Takeout,OutdoorSeating,Seating,ParkingAvailabl...","smokedmeat,jambalaya,cornbread,beignets,cubans...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,1
1,395,Restaurant,Greek Islands Restaurant,4.5,589.0,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,mediterranean greek canadian,highcost,Greek Islands Restaurant,"great friendly staff,good price,good wine beer...",NaN,NaN,NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Tuesday,11:00,22:00|Wednesday,11:00,22:00|Thur...","Mediterranean,Greek,Canadian,Lunch,Dinner,Afte...","Takeout,Reservations,Seating,ParkingAvailable,...","greeksalad,roastlamb,saganaki,soup,pitabread,p...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,46
2,394,Restaurant,Greek Islands Restaurant,4.5,589.0,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,mediterranean greek canadian,highcost,Greek Islands Restaurant,"food excellent staff,special occasion food,awe...",NaN,"exquisite chinese culinary restaurant ,delicio...",NaN,Greek Islands Restaurant,Twin brothers George and Gus Sahamis migrated ...,NaN,"2686 Langdon St, Abbotsford, British Columbia ...",(604) 850-1613,gi1@greekislandsrestaurants.com,http://greekislandsrestaurants.com/greek-islan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https:/

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

In [45]:
        data = d
        data['Cuisine'] = data['Cuisine'].fillna(" ")
        import re
        def remove_punc(text):
            text = text.replace(',', ' ')
            return text

        data['Cuisine'] = data['Cuisine'].apply(lambda x : remove_punc(x))
        tfv = TfidfVectorizer(min_df=3,  max_features=None, 
                    strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 3),
                    stop_words = 'english')

        tfv_matrix = tfv.fit_transform(data['Cuisine'])  
        df = pd.DataFrame(tfv_matrix[0].T.todense(), index=tfv.get_feature_names(), columns=["TF-IDF"])
        df = df.sort_values('TF-IDF', ascending=False)
        Kmeans = KMeans(n_clusters=50,init='k-means++',random_state=0)
        kmeans.fit(tfv_matrix)
        data['clusters'] = kmeans.labels_
        def cluster_predict(str_input):
            Y = tfv.transform(str_input)
            prediction = kmeans.predict(Y)
            return prediction
        user = fil_user
        res = fil_res
        users_id = list(user['User_id'])
        new_user = usid
        fav_cus = fav_cuisine
        if new_user not in users_id:
                print('new user')
                f = fav_cus
                prediction = cluster_predict(f)
                data1 =  data[data['clusters'] == prediction[0]]
                list2 = list(data1['Name'])
                result2 = list2[:15]
        else:
            print('existing user')
            x = list(user[(user['User_id'] == new_user) & (user['Rating_id'] > 3.0) ]['Restaurant_id'])
            my_favorite_restaurants = [(res[res['resId'] == i]['title'].values)[0] for i in x]
            result2=my_favorite_restaurants[:3]
            cus = [(data[data['Name'] == i]['Cuisine'].values)[0] for i in result2[:3]]
            prediction = cluster_predict(cus)
            data1 =  data[data['clusters'] == prediction[0]]
            list2 = list(data['Name'])
            result2 += list2


new user


In [46]:
result2

['Mr India Restaurant',
 'Mr India Restaurant',
 'Mr India Restaurant',
 'Mr India Restaurant',
 'Bayside Grill',
 'Taj Indian Bistro',
 'Taj Indian Bistro',
 'Chakra Indian Restaurant',
 'Sapphire Bistro Banquet',
 'Sapphire Bistro Banquet',
 'Sapphire Bistro Banquet',
 'White Rock Indian Flavours',
 'Curry Express',
 'Curry Express',
 'Curry Express']

In [47]:
data1

,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,...,time_f,time_f1,Cuisine,Price,Restaurant,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes,Title,Description,Tagline,Google Address,Phone,Email,Website,Twitter,Facebook,Linkedin,Google_Plus,Youtube,Instagram,Youtube Video URL,Price Status ($-moderatemoderate),Price From,Price To,Claim Status,Faq Question (sep. by pipe sign |),Faq Answer (sep. by pipe sign |),Gallery,Pricing Plan ID,"Business Hours (Day,OpenTime,CloseTime)",Category,Features,Tags (Keywords),Location,Unnamed: 29,Restauraunt Links,city,clusters
114,Restaurant,Mr India Restaurant,4.0,14.0,"[{'days': 'Wed - Mon', 'times': ['10:00 AM - 1...",49.048660,-122.339240,-,"31940 South Fraser Way, Abbotsford, British Co...",+1 604-870-4898,4.5,4.0,4.0,0.0,3.721848,3.0,1.0,10.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,indian asian,highcost,Mr India Restaurant,"best service,clean place,outstanding cake,happ...",processed chicken cube,"excellent dinner,awesome pizza","a little pricey but nor too bad,brutal",Noodle Road,NaN,NaN,"24-31940 South Fraser Way, Abbotsford, British...",(604) 557-0600,support@noddleroad.ca,http://www.noodleroad.ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,24:00,21:00|Monday,11:00,21:00|Tuesday,...","Sushi,Asian,Korean,Japanese,Lunch,Dinner,Veget...","Takeout,Seating,ParkingAvailable,WheelchairAcc...","noodles,sushi,asiandishes,complimentaryappetiz...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,11
115,Restaurant,Mr India Restaurant,4.0,14.0,"[{'days': 'Wed - Mon', 'times': ['10:00 AM - 1...",49.048660,-122.339240,-,"31940 South Fraser Way, Abbotsford, British Co...",+1 604-870-4898,4.5,4.0,4.0,0.0,3.721848,3.0,1.0,10.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,indian asian,highcost,Mr India Restaurant,"best service,clean place,outstanding cake,happ...",processed chicken cube,"excellent dinner,awesome pizza","a little pricey but nor too bad,brutal",Mr India Restaurant,NaN,NaN,"31940 South Fraser Way, Abbotsford, British Co...",(604) 870-4898,NaN,https://mr-india-meats-restaurant.business.site/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid expensive,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Wednesday,10:00,22:00|Thursday,10:00,22:00|Fri...","Indian,Asian,Dinner,Lunch,","Seating,WheelchairAccessible",",best service,clean place,outstanding cake,hap...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,11
116,Restaurant,Mr India Restaurant,4.0,14.0,"[{'days': 'Wed - Mon', 'times': ['10:00 AM - 1...",49.048660,-122.339240,-,"31940 South Fraser Way, Abbotsford, British Co...",+1 604-870-4898,4.5,4.0,4.0,0.0,3.721848,3.0,1.0,10.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,indian asian,highcost,Mr India Restaurant,"best service,clean place,outstanding cake,happ...",processed chicken cube,"excellent dinner,awesome pizza","a little pricey but nor too bad,brutal",Super Punjab Sweet House,NaN,NaN,"31940 South Fraser Way, Abbotsford, British Co...",(604) 504-2828,NaN,http://superpunjabsweethouseabbotsford.gastrob...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclaimed,NaN,NaN,https://media-cdn.tripadvisor.com/media/photo-...,NaN,"Sunday,10:00,19:00|Monday,10:00,19:00|Tuesday,...",",Lunch,Dinner,",NaN,",good variety samosa,many people,fantastic sam...",Abbotsford,NaN,https://www.tripadvisor.ca/Restaurant_Review-g...,Abbotsford,11
117,Restaurant,Mr India Restaurant,4.0,14.0,"[{'days': 'Wed - Mon', 'times': [

In [51]:
kmeans = KMeans(n_clusters=50,init='k-means++',random_state=0)
kmeans.fit(tfv_matrix)
data['clusters'] = kmeans.labels_
def cluster_predict(str_input):
    Y = tfv.transform(str_input)
    prediction = kmeans.predict(Y)
    return prediction
user = fil_user
res = fil_res
users_id = list(user['User_id'])
new_user = usid
fav_cus = fav_cuisine
if new_user not in users_id:
  print('new user')
  f = fav_cus
  prediction = cluster_predict(f)
  data1 =  data[data['clusters'] == prediction[0]]
  list2 = list(data1['Name'])
  result2 = list2[:15]
else:
  print('existing user')
  x = list(user[(user['User_id'] == new_user) & (user['Rating_id'] > 3.0) ]['Restaurant_id'])
  my_favorite_restaurants = [(res[res['resId'] == i]['title'].values)[0] for i in x]
  result2=my_favorite_restaurants[:3]
  cus = [(data[data['Name'] == i]['Cuisine'].values)[0] for i in result2[:3]]
  prediction = cluster_predict(cus)
  data1 =  data[data['clusters'] == prediction[0]]
  list2 = list(data['Name'])
  result2 += list2


new user
